In [1]:
#model
import pandas as pd
import datetime as dt
import math
import numpy as np

import yfinance as yf
from nsepy import get_history
import pandas_datareader.data as web

from sklearn import preprocessing, svm
import sklearn.model_selection as skms
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')


start = dt.datetime(2000,1,1)
end = dt.datetime.now()
#df = web.DataReader('FB', 'yahoo', start, end)
df = get_history("SBIN", start, end)


#volatility;;percentage change
df['HL_PCT'] = ( (df['High']-df['Close']) / df['Close'] ) * 100
df['PCT_change'] = ( (df['Close']-df['Open']) / df['Open'] ) * 100
#needed attributes
df = df[['Close', 'HL_PCT', 'PCT_change', 'Volume']]
forecast_col = 'Close'
df.fillna(-99999,inplace = True) #making them an outlier

forecast_out = int(math.ceil(0.01*len(df))) ##last ten days
#print(forecast_out)
df['label'] = df[forecast_col].shift(-forecast_out)


#features apart from label;;CAPITAL X
X = np.array(df.drop(['label'],1)) #returns a new df
X = preprocessing.scale(X)
X = X[:-forecast_out]


X_lately = X[-forecast_out:]


df.dropna(inplace=True)
y = np.array(df['label'])
y = np.array(df['label'])

X_train, X_test, y_train, y_test = skms.train_test_split(X,y,test_size = 0.3)

clf = LinearRegression(n_jobs = -1) #runs as many jobs at a time as possible
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
#print(accuracy)
#clf2 = svm.SVR()
#clf2.fit(X_train,y_train)
#accuracy2 = clf2.score(X_test,y_test)
#print(accuracy2)


forecast_set = clf.predict(X_lately)

print(forecast_set, accuracy, forecast_out)
df['Forecast'] = np.nan

##for x-axis date;; since we droped it for calculation
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day
for i in forecast_set:
    next_date = dt.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]   

#df['Close'].plot()
#df['Forecast'].plot()
#plt.legend(loc=4)
#plt.xlabel('Date')
#plt.ylabel('Price')
#plt.show()

#print(df.head(5))


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
#GUI
import tkinter as tk

from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.

from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure

import numpy as np

df2 = df
root = tk.Tk()
root.wm_title("Embedding in Tk")

fig = Figure(figsize=(15, 6), dpi=100)

ax2 = fig.add_subplot(111)

df2['Close'].plot(kind='line', linewidth = .8,legend =True,ax = ax2, color = '#3385ff',fontsize = 10)
df2['Forecast'].plot(kind='line', linewidth = .7,legend =True,ax = ax2, color = '#00e600',fontsize=10)

ax2.fill_between(df2.index, pricemin, df['Forecast'], facecolor='#33ff33', alpha=0.5)
ax2.fill_between(df2.index, pricemin, df['Close'], facecolor='#3385ff', alpha=0.5)

canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
canvas.draw()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

toolbar = NavigationToolbar2Tk(canvas, root)
toolbar.update()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)


def on_key_press(event):
    print("you pressed {}".format(event.key))
    key_press_handler(event, canvas, toolbar)


canvas.mpl_connect("key_press_event", on_key_press)


def _quit():
    root.quit()     
    root.destroy()  
                    

button = tkinter.Button(master=root, text="Quit", command=_quit, color = 'r')
button.pack(side=tkinter.BOTTOM)

tkinter.mainloop()


In [ ]:
#option
import tkinter as tk


def sel():
    
    selection = "You selected the option " + str(var.get())
    label4 = tk.Label(root,text = selection)
    canvas1.create_window(320, 100, window=label4)
    #label.config()

#if __name__ == '__main__':
root = tk.Tk()
canvas1 = tk.Canvas(root, width = 400, height = 400, relief = 'raised')
canvas1.pack()
var = tk.IntVar()
var.set(1)

R1 = tk.Radiobutton(root, text="Option 1", variable=var, value=1,command=sel,activeforeground='blue')
canvas1.create_window(140, 60, window=R1)

R2 = tk.Radiobutton(root, text="Option 2", variable=var, value=2,command=sel,activeforeground='blue')
canvas1.create_window(140, 80, window=R2)

R3 = tk.Radiobutton(root, text="Option 3", variable=var, value=3,command=sel,activeforeground='blue')
canvas1.create_window(140, 100, window=R3)

root.mainloop()

In [ ]:
##GUI Page 1

import tkinter as tk

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 600, height = 500, relief = 'raised')
canvas1.pack()

label2 = tk.Label(root, text='Do you want to Include Additional Options/Include Intuition ?')
label2.config(font=('helvetica',15))
canvas1.create_window(320, 200, window=label2)

#entry1 = tk.Entry (root,width=35) 
#canvas1.create_window(320, 220, window=entry1)

def Yes():
    canvas1.delete('all')
    label3 = tk.Label(root, text='What do you think will be the \nGeneral direction for the upcoming days?')
    label3.config(font=('helvetica',15))
    canvas1.create_window(320, 100, window=label3)
    button3 = Radiobutton(root, text="Up", variable=var, value=1,command=sel)
    canvas1.create_window(250, 260, window=button3)

def Options():
    canvas1.delete('all')
    label4 = tk.Label(root, text='Select the options that you also want along with the prediction :')
    label4.config(font=('helvetica',15))
    canvas1.create_window(320, 100, window=label4)
    
    
button1 = tk.Button(root, text='YES', command=Yes, bg='green', fg='white', font=('helvetica', 11, 'bold'))
canvas1.create_window(250, 260, window=button1)

button2 = tk.Button(root, text='NO', command=Options, bg='red', fg='white', font=('helvetica', 11, 'bold'))
canvas1.create_window(340, 260, window=button2)

root.mainloop()
